# Generate mni subject

1. Imports
2. Download mni template
3. Apply reconall
4. Build surfaces from aseg

In [1]:
from nilearn.datasets import load_mni152_template,get_data_dirs, fetch_icbm152_2009
from nipype.interfaces.freesurfer.preprocess import ReconAll
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.utility import Function
import nipype.interfaces.freesurfer as fs
from nipype.interfaces.io import FreeSurferSource
import os

/Users/spherik/anaconda3/envs/brain_env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data_dir = './mni-template/'
template = fetch_icbm152_2009(data_dir=data_dir)
print(template['t1'])



Dataset created in ./mni-template/icbm152_2009



Downloaded 63116438 of 63116438 bytes (100.0%,    0.0s remaining) ...done. (30 seconds, 0 min)
Extracting data from ./mni-template/icbm152_2009/6f7789dc5278e29886939666a6657dbc/mni_icbm152_nlin_sym_09a_nifti.zip...

./mni-template/icbm152_2009/mni_icbm152_nlin_sym_09a/mni_icbm152_t1_tal_nlin_sym_09a.nii


.. done.


In [3]:
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

reconall = ReconAll()
reconall.inputs.subject_id = 'mni'
reconall.inputs.directive = 'all'
reconall.inputs.subjects_dir = data_dir
reconall.inputs.T1_files = template['t1']
reconall.inputs.openmp = 2
reconall.inputs.parallel = True
reconall.run()

OSError: No command "recon-all" found on host Tonis-MacBook-Pro.local. Please check that the corresponding package is installed.

In [3]:
NAMELIST = [
#[1,   'Left-Cerebral-Exterior',                  70,  130, 180, 0],
[2,   'Left-Cerebral-White-Matter',              245, 245, 245, 0],
[3,   'Left-Cerebral-Cortex',                    205, 62,  78,  0],
[4,   'Left-Lateral-Ventricle',                  120, 18,  134, 0],
[5,   'Left-Inf-Lat-Vent',                       196, 58,  250, 0],
#[6,   'Left-Cerebellum-Exterior',                0,   148, 0,   0],
[7,   'Left-Cerebellum-White-Matter',            220, 248, 164, 0],
[8,   'Left-Cerebellum-Cortex',                 230, 148, 34,  0],
[10,  'Left-Thalamus-Proper',                   0,   118, 14,  0],
[11,  'Left-Caudate',                            122, 186, 220, 0],
[12,  'Left-Putamen',                            236, 13,  176, 0],
[13,  'Left-Pallidum',                           12,  48,  255, 0],
[14,  '3rd-Ventricle',                           204, 182, 142, 0],
[15,  '4th-Ventricle',                           42,  204, 164, 0],
[16,  'Brain-Stem',                              119, 159, 176, 0],
[17,  'Left-Hippocampus',                        220, 216, 20,  0],
[18,  'Left-Amygdala',                           103, 255, 255, 0],
#[19,  'Left-Insula',                             80,  196, 98,  0],
[26,  'Left-Accumbens-area',                     255, 165, 0,   0],
[28,  'Left-VentralDC',                          165, 42,  42,  0],
#[40,  'Right-Cerebral-Exterior',                 70,  130, 180, 0],
[41,  'Right-Cerebral-White-Matter',             245, 245, 245, 0],
[42,  'Right-Cerebral-Cortex',                   205, 62,  78, 0],
[43,  'Right-Lateral-Ventricle',                 120, 18,  134, 0],
[44,  'Right-Inf-Lat-Vent',                      196, 58,  250, 0],
#[45,  'Right-Cerebellum-Exterior',               0,   148, 0,   0],
[46,  'Right-Cerebellum-White-Matter',           220, 248, 164, 0],
[47,  'Right-Cerebellum-Cortex',                 230, 148, 34,  0],
[49, 'Right-Thalamus-Proper',                  0,   118, 14,  0],
[50,  'Right-Caudate',                           122, 186, 220, 0],
[51,  'Right-Putamen',                           236, 13,  176, 0],
[52,  'Right-Pallidum',                          13,  48,  255, 0],
[53,  'Right-Hippocampus',                       220, 216, 20,  0],
[54,  'Right-Amygdala',                          103, 255, 255, 0],
#[55,  'Right-Insula',                            80,  196, 98,  0],
[58,  'Right-Accumbens-area',                    255, 165, 0,   0],
[60,  'Right-VentralDC',                         165, 42,  42,  0],
[251, 'CC_Posterior',                            0,   0,   64,  0],
[252, 'CC_Mid_Posterior',                        0,   0,   112, 0],
[253, 'CC_Central',                              0,   0,   160, 0],
[254, 'CC_Mid_Anterior',                         0,   0,   208, 0],
[255, 'CC_Anterior',                             0,   0,   255, 0]]

In [13]:
wf = Workflow(name="generate_surfaces")

fssource = Node(interface=FreeSurferSource(subjects_dir=os.path.abspath(data_dir),subject_id='mni'), name='fssource')
pretess = Node(interface=fs.MRIPretess(subjects_dir=os.path.abspath(data_dir)), name='pretess')
tess =  Node(interface=fs.MRITessellate(subjects_dir=os.path.abspath(data_dir)), name='tess')
smooth =  Node(interface=fs.SmoothTessellation(subjects_dir=os.path.abspath(data_dir)), name='smooth')
convert = Node(interface=fs.MRIsConvert(subjects_dir=os.path.abspath(data_dir),), name='convert')

wf.connect(fssource, 'aseg', pretess, 'in_filled')
wf.connect(fssource, 'norm', pretess, 'in_norm')
wf.connect(pretess, 'out_file', tess, 'in_file')
wf.connect(tess, 'surface', smooth, 'in_file')
wf.connect(smooth, 'surface', convert, 'in_file')

In [15]:
for l in NAMELIST:
    # Pre tessellation
    pretess.inputs.label = l[0]
    pretess.inputs.out_file = os.path.join(os.path.abspath(data_dir), 'mni', 'tmp', l[1]+'_filled.mgz')
    pretess.outputs
    # Tesselation
    tess.inputs.label_value = l[0]
    tess.inputs.out_file = os.path.join(os.path.abspath(data_dir), 'mni', 'tmp',l[1]+'_nonsmooth' )
    # Smoothing
    smooth.inputs.out_file = os.path.join(os.path.abspath(data_dir), 'mni', 'tmp',l[1] )
    # Convert
    convert.inputs.out_file = os.path.join(os.path.abspath(data_dir), 'mni', 'tmp', l[1]+'.surf.gii')
    # Run pipeline
    wf.run()      

    os.remove(pretess.inputs.out_file)
    os.remove(tess.inputs.out_file)
    os.remove(smooth.inputs.out_file)

180419-09:34:13,60 workflow INFO:
	 Workflow generate_surfaces settings: ['check', 'execution', 'logging', 'monitoring']
180419-09:34:13,69 workflow INFO:
	 Running serially.
180419-09:34:13,71 workflow INFO:
	 [Node] Setting-up "generate_surfaces.fssource" in "/var/folders/kf/218x3zrs5yd240h419ybsryh0000gq/T/tmpl8z2hylz/generate_surfaces/fssource".
180419-09:34:13,77 workflow INFO:
	 [Node] Running "fssource" ("nipype.interfaces.io.FreeSurferSource")
180419-09:34:13,104 workflow INFO:
	 [Node] Finished "generate_surfaces.fssource".
180419-09:34:13,107 workflow INFO:
	 [Node] Setting-up "generate_surfaces.pretess" in "/var/folders/kf/218x3zrs5yd240h419ybsryh0000gq/T/tmpya_5ai_s/generate_surfaces/pretess".
180419-09:34:13,121 workflow INFO:
	 [Node] Running "pretess" ("nipype.interfaces.freesurfer.utils.MRIPretess"), a CommandLine Interface with command:
mri_pretess /Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/mri/aseg.mgz 2 /Users/tgurgui/Documents/DevLibs/python_

In [6]:
fssource2 = FreeSurferSource(subjects_dir=data_dir,subject_id='mni')
print(fssource2.inputs)
res = fssource2.run()
res.outputs


hemi = both
ignore_exception = False
subject_id = mni
subjects_dir = ./mni-template/




BA_stats = <undefined>
T1 = /Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/mri/T1.mgz
annot = ['/Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/label/lh.aparc.a2009s.annot', '/Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/label/lh.aparc.annot', '/Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/label/lh.aparc.DKTatlas.annot', '/Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/label/lh.BA_exvivo.annot', '/Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/label/lh.BA_exvivo.thresh.annot', '/Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/label/rh.aparc.a2009s.annot', '/Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/label/rh.aparc.annot', '/Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/label/rh.aparc.DKTatlas.annot', '/Users/tgurgui/Documents/DevLibs/python_for_brain/mni-template/mni/label/rh.BA_exvivo.annot', '/Users/tgurgu

In [16]:
template['t1']

'./mni-template/icbm152_2009/mni_icbm152_nlin_sym_09a/mni_icbm152_t1_tal_nlin_sym_09a.nii'